In [2]:
import pandas as pd
from country_converter import CountryConverter

path1 = "C:\\Users\\Brady\\Dropbox\\Econometrics\\Conflict Frequency and Cultural Values\\Datasets\\conflict.dta"

df1 = pd.read_stata(path1)


In [ ]:
path2 = "C:\\Users\\Brady\\Dropbox\\Econometrics\\Conflict Frequency and Cultural Values\\Datasets\\survey_working.dta"
df2 = pd.read_stata(path2, convert_categoricals=False)
df2.B_COUNTRY


In [ ]:
df2 = df2[df2.B_COUNTRY != 909]
df2.B_COUNTRY

In [ ]:
df2['Country'] = df2.B_COUNTRY.apply(lambda code: CountryConverter().convert(code, to='name_short'))
df2['Country']


In [ ]:
df2.to_stata("Datasets/survey_working2.dta")

In [17]:
df1['Country'] = df1.side_a.str.replace('Government of ', '')
df1['Country'] = df1.Country.str.replace(' of America', '')
df1.Country

0           Mexico
1           Mexico
2       Mozambique
3       Mozambique
4       Mozambique
           ...    
2621        Angola
2622        Angola
2623        Angola
2624        Angola
2625        Angola
Name: Country, Length: 2626, dtype: object

In [18]:
df1.start_date = pd.to_datetime(df1.start_date)
df1.ep_end_date = pd.to_datetime(df1.ep_end_date)
df1['Duration'] = (df1.ep_end_date - df1.start_date).dt.days
conflict_duration = df1.groupby(['Country', 'conflict_id'])['Duration'].max().reset_index()
total_conflict_duration = conflict_duration.groupby('Country')['Duration'].sum().reset_index()
total_conflict_duration

,Country,Duration
0,Afghanistan,38.0
1,Albania,70.0
2,Algeria,13130.0
3,Angola,21110.0
4,Argentina,8307.0
...,...,...
115,Uruguay,993.0
116,Uzbekistan,1991.0
117,Venezuela,11135.0
118,Yemen (North Yemen),14637.0


In [ ]:
merged = pd.merge(df2, total_conflict_duration, on='Country', how='left', indicator=True)

In [ ]:
merged.to_stata("Datasets/sur_con_m.dta")

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'total_country_duration' is the DataFrame with total conflict durations for each country
# Sort the DataFrame by 'Country' in alphabetical order
sorted_duration = total_conflict_duration.sort_values(by='Country')

# Plotting the bar chart with increased figure size and rotated x-axis labels
plt.figure(figsize=(14, 8))
bars = plt.bar(sorted_duration['Country'], sorted_duration['Duration'])
plt.xlabel('Country')
plt.ylabel('Total Conflict Duration (days)')
plt.title('Total Conflict Duration by Country')
plt.xticks(rotation=45, ha='right')  # Rotate and adjust alignment of x-axis labels
plt.tight_layout()

# Add data labels above the bars for better visibility
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom', color='black')

# Show the plot
plt.show()


In [8]:
path_m = "C:\\Users\\Brady\\Dropbox\\Econometrics\\Conflict Frequency and Cultural Values\\sur_con_m_working2.dta"
df_m = pd.read_stata(path_m, convert_categoricals=False)
df_m.Q266

C:\Users\Brady\AppData\Local\Temp\ipykernel_13044\302655068.py:2: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df_m = pd.read_stata(path_m, convert_categoricals=False)


0        862
1        400
2        531
3         56
4        400
        ... 
45437    368
45438    862
45439    404
45440    504
45441    862
Name: Q266, Length: 45442, dtype: int16

In [ ]:
df_m['c_origin'] = df_m['Q266'].apply(lambda code: CountryConverter().convert(code, to='name_short'))
df_m['c_origin']


In [33]:
merge_2 = pd.merge(df_m, total_conflict_duration, on='Country', how='left', indicator=True)


In [35]:
merge_2.to_stata("Datasets\sub_con_m_origin.dta")